# ItaData2024

## propositional analysis intersting rules comparision

In [1]:
using Pkg
Pkg.activate(".")
using MLJ, ModalDecisionTrees
using SoleDecisionTreeInterface, Sole, SoleData
using CategoricalArrays
using DataFrames, JLD2, CSV
using Audio911
using Random
using StatsBase, Catch22
using Test
using Plots

sr = 8000

avail_exp = [:Pneumonia, :Bronchiectasis, :COPD, :URTI, :Bronchiolitis]
findhealthy = y -> findall(x -> x == "Healthy", y)
findsick = y -> findall(x -> x == String(experiment), y)

color_code = Dict(:red => 31, :green => 32, :yellow => 33, :blue => 34, :magenta => 35, :cyan => 36);
r_select = r"\e\[\d+m(.*?)\e\[0m";
catch9_f = ["max", "min", "mean", "med", "std", "bsm", "bsd", "qnt", "3ac"]
catch9 = [
    maximum,
    minimum,
    StatsBase.mean,
    median,
    std,
    Catch22.SB_BinaryStats_mean_longstretch1,
    Catch22.SB_BinaryStats_diff_longstretch0,
    Catch22.SB_MotifThree_quantile_hh,
    Catch22.SB_TransitionMatrix_3ac_sumdiagcov,
]
;

  Activating project at `~/Documents/Aclai/audio-rules2024`


### Pneumonia
scale: semitones without mfcc

In [2]:
experiment = :Pneumonia
# experiment = :Bronchiectasis
# experiment = :COPD
# experiment = :URTI
# experiment = :Bronchiolitis

scale = :semitones
# scale = :mel_htk

featset = ()
# featset = (:mfcc,)

audioparams = (
    sr = sr,
    nfft = 512,
    mel_scale = scale, # :mel_htk, :mel_slaney, :erb, :bark, :semitones, :tuned_semitones
    mel_nbands = scale == :semitones ? 14 : 26,
    mfcc_ncoeffs = scale == :semitones ? 7 : 13,
    mel_freqrange = (300, round(Int, sr / 2)),
    mel_dbscale = :mfcc in featset ? false : true,
    audio_norm = true,
)

@assert experiment in avail_exp "Unknown type of experiment: $experiment."
ds_path = "/datasets/respiratory_Healthy_" * String(experiment)
filename = "/datasets/itadata2024_" * String(experiment) * "_files"

destpath = "results/propositional/$scale"
:mfcc in featset ? destpath *= "_mfcc/" : destpath *= "/"
jld2file = destpath * "/itadata2024_" * String(experiment) * "_" * String(scale) * ".jld2"
dsfile = destpath * "/ds_test_" * String(experiment) * "_" * String(scale) * ".jld2"

d = jldopen(string((@__DIR__), ds_path, ".jld2"))
x, y = d["dataframe_validated"]
@assert x isa DataFrame
close(d)

freq = round.(Int, afe(x[1, :audio]; featset=(:get_only_freqs), audioparams...))

variable_names = vcat([
    vcat(
        ["\e[$(color_code[:yellow])m$j(mel$i=$(freq[i])Hz)\e[0m" for i in 1:audioparams.mel_nbands],
        :mfcc in featset ? ["\e[$(color_code[:red])m$j(mfcc$i)\e[0m" for i in 1:audioparams.mfcc_ncoeffs] : String[],
        :f0 in featset ? ["\e[$(color_code[:green])m$j(f0)\e[0m"] : String[],
        "\e[$(color_code[:cyan])m$j(cntrd)\e[0m", "\e[$(color_code[:cyan])m$j(crest)\e[0m",
        "\e[$(color_code[:cyan])m$j(entrp)\e[0m", "\e[$(color_code[:cyan])m$j(flatn)\e[0m", "\e[$(color_code[:cyan])m$j(flux)\e[0m",
        "\e[$(color_code[:cyan])m$j(kurts)\e[0m", "\e[$(color_code[:cyan])m$j(rllff)\e[0m", "\e[$(color_code[:cyan])m$j(skwns)\e[0m",
        "\e[$(color_code[:cyan])m$j(decrs)\e[0m", "\e[$(color_code[:cyan])m$j(slope)\e[0m", "\e[$(color_code[:cyan])m$j(sprd)\e[0m"
    )
    for j in catch9_f
]...)

@info("Load dataset...")
d = jldopen(dsfile)
X_test = d["X_test"]
y_test = d["y_test"]
close(d)
d = jldopen(jld2file)
sole_dt = d["sole_dt"]
close(d)

interesting_rules = listrules(sole_dt,
	min_lift = 1.0,
	# min_lift = 2.0,
	min_ninstances = 0,
	min_coverage = 0.10,
	normalize = true,
    variable_names_map=variable_names
);
map(r->(consequent(r), readmetrics(r)), interesting_rules)

readmetrics.(interesting_rules)

printmodel.(
    sort(interesting_rules, by = readmetrics); 
    show_metrics = (; round_digits = 2, additional_metrics = (; height = r->SoleLogics.height(antecedent(r)))), 
    syntaxstring_kwargs = (; threshold_digits = 2), 
    variable_names_map=variable_names
);

┌ Info: Load dataset...
└ @ Main /home/paso/Documents/Aclai/audio-rules2024/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_W3sZmlsZQ==.jl:53


▣ (std(skwns) < 16.88) ∧ (std(mel10=1687Hz) ≥ 0.54) ∧ (bsd(mel14=3366Hz) ≥ 3.5) ∧ (3ac(mel8=1194Hz) ≥ 0.01)  ↣  Healthy : (ninstances = 101, ncovered = 12, coverage = 0.12, confidence = 0.92, lift = 1.68, natoms = 4, height = 3)
▣ (std(skwns) < 16.88) ∧ (std(mel10=1687Hz) < 0.54) ∧ (min(mel8=1194Hz) < -4.2) ∧ (std(mel8=1194Hz) ≥ 0.19) ∧ (max(mel6=845Hz) ≥ -3.58) ∧ (std(mel6=845Hz) ≥ 0.48) ∧ (max(decrs) < 0.93)  ↣  Pneumonia : (ninstances = 101, ncovered = 15, coverage = 0.15, confidence = 0.73, lift = 1.61, natoms = 7, height = 6)
▣ (std(skwns) < 16.88) ∧ (std(mel10=1687Hz) < 0.54) ∧ (min(mel8=1194Hz) < -4.2) ∧ (std(mel8=1194Hz) < 0.19)  ↣  Pneumonia : (ninstances = 101, ncovered = 17, coverage = 0.17, confidence = 1.0, lift = 2.2, natoms = 4, height = 3)
▣ (std(skwns) ≥ 20.37) ∧ (std(mel9=1419Hz) ≥ 0.14) ∧ (med(mel9=1419Hz) < -3.28)  ↣  Healthy : (ninstances = 101, ncovered = 22, coverage = 0.22, confidence = 0.95, lift = 1.75, natoms = 3, height = 2)


### Pneumonia
scale: semitones with mfcc

In [3]:
experiment = :Pneumonia
# experiment = :Bronchiectasis
# experiment = :COPD
# experiment = :URTI
# experiment = :Bronchiolitis

scale = :semitones
# scale = :mel_htk

# featset = ()
featset = (:mfcc,)

audioparams = (
    sr = sr,
    nfft = 512,
    mel_scale = scale, # :mel_htk, :mel_slaney, :erb, :bark, :semitones, :tuned_semitones
    mel_nbands = scale == :semitones ? 14 : 26,
    mfcc_ncoeffs = scale == :semitones ? 7 : 13,
    mel_freqrange = (300, round(Int, sr / 2)),
    mel_dbscale = :mfcc in featset ? false : true,
    audio_norm = true,
)

@assert experiment in avail_exp "Unknown type of experiment: $experiment."
ds_path = "/datasets/respiratory_Healthy_" * String(experiment)
filename = "/datasets/itadata2024_" * String(experiment) * "_files"

destpath = "results/propositional/$scale"
:mfcc in featset ? destpath *= "_mfcc/" : destpath *= "/"
jld2file = destpath * "/itadata2024_" * String(experiment) * "_" * String(scale) * ".jld2"
dsfile = destpath * "/ds_test_" * String(experiment) * "_" * String(scale) * ".jld2"

d = jldopen(string((@__DIR__), ds_path, ".jld2"))
x, y = d["dataframe_validated"]
@assert x isa DataFrame
close(d)

freq = round.(Int, afe(x[1, :audio]; featset=(:get_only_freqs), audioparams...))

variable_names = vcat([
    vcat(
        ["\e[$(color_code[:yellow])m$j(mel$i=$(freq[i])Hz)\e[0m" for i in 1:audioparams.mel_nbands],
        :mfcc in featset ? ["\e[$(color_code[:red])m$j(mfcc$i)\e[0m" for i in 1:audioparams.mfcc_ncoeffs] : String[],
        :f0 in featset ? ["\e[$(color_code[:green])m$j(f0)\e[0m"] : String[],
        "\e[$(color_code[:cyan])m$j(cntrd)\e[0m", "\e[$(color_code[:cyan])m$j(crest)\e[0m",
        "\e[$(color_code[:cyan])m$j(entrp)\e[0m", "\e[$(color_code[:cyan])m$j(flatn)\e[0m", "\e[$(color_code[:cyan])m$j(flux)\e[0m",
        "\e[$(color_code[:cyan])m$j(kurts)\e[0m", "\e[$(color_code[:cyan])m$j(rllff)\e[0m", "\e[$(color_code[:cyan])m$j(skwns)\e[0m",
        "\e[$(color_code[:cyan])m$j(decrs)\e[0m", "\e[$(color_code[:cyan])m$j(slope)\e[0m", "\e[$(color_code[:cyan])m$j(sprd)\e[0m"
    )
    for j in catch9_f
]...)

@info("Load dataset...")
d = jldopen(dsfile)
X_test = d["X_test"]
y_test = d["y_test"]
close(d)
d = jldopen(jld2file)
sole_dt = d["sole_dt"]
close(d)

interesting_rules = listrules(sole_dt,
	min_lift = 1.0,
	# min_lift = 2.0,
	min_ninstances = 0,
	min_coverage = 0.10,
	normalize = true,
    variable_names_map=variable_names
);
map(r->(consequent(r), readmetrics(r)), interesting_rules)

readmetrics.(interesting_rules)

printmodel.(
    sort(interesting_rules, by = readmetrics); 
    show_metrics = (; round_digits = 2, additional_metrics = (; height = r->SoleLogics.height(antecedent(r)))), 
    syntaxstring_kwargs = (; threshold_digits = 2), 
    variable_names_map=variable_names
);

┌ Info: Load dataset...
└ @ Main /home/paso/Documents/Aclai/audio-rules2024/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_W5sZmlsZQ==.jl:53


▣ (min(mfcc2) ≥ 1.5) ∧ (min(mfcc3) ≥ -0.53) ∧ (min(mel14=3366Hz) ≥ 0.0) ∧ (med(cntrd) < 19.89) ∧ (3ac(mfcc4) ≥ 0.0)  ↣  Pneumonia : (ninstances = 101, ncovered = 27, coverage = 0.27, confidence = 0.96, lift = 1.91, natoms = 5, height = 4)
▣ (min(mfcc2) < 1.5) ∧ (max(mfcc4) ≥ 0.56) ∧ (mean(mfcc5) < 0.48) ∧ (med(mel11=2005Hz) < 0.0)  ↣  Healthy : (ninstances = 101, ncovered = 37, coverage = 0.37, confidence = 0.86, lift = 1.75, natoms = 4, height = 3)


### Pneumonia
scale: mel _htk without mfcc

In [4]:
experiment = :Pneumonia
# experiment = :Bronchiectasis
# experiment = :COPD
# experiment = :URTI
# experiment = :Bronchiolitis

# scale = :semitones
scale = :mel_htk

featset = ()
# featset = (:mfcc,)

audioparams = (
    sr = sr,
    nfft = 512,
    mel_scale = scale, # :mel_htk, :mel_slaney, :erb, :bark, :semitones, :tuned_semitones
    mel_nbands = scale == :semitones ? 14 : 26,
    mfcc_ncoeffs = scale == :semitones ? 7 : 13,
    mel_freqrange = (300, round(Int, sr / 2)),
    mel_dbscale = :mfcc in featset ? false : true,
    audio_norm = true,
)

@assert experiment in avail_exp "Unknown type of experiment: $experiment."
ds_path = "/datasets/respiratory_Healthy_" * String(experiment)
filename = "/datasets/itadata2024_" * String(experiment) * "_files"

destpath = "results/propositional/$scale"
:mfcc in featset ? destpath *= "_mfcc/" : destpath *= "/"
jld2file = destpath * "/itadata2024_" * String(experiment) * "_" * String(scale) * ".jld2"
dsfile = destpath * "/ds_test_" * String(experiment) * "_" * String(scale) * ".jld2"

d = jldopen(string((@__DIR__), ds_path, ".jld2"))
x, y = d["dataframe_validated"]
@assert x isa DataFrame
close(d)

freq = round.(Int, afe(x[1, :audio]; featset=(:get_only_freqs), audioparams...))

variable_names = vcat([
    vcat(
        ["\e[$(color_code[:yellow])m$j(mel$i=$(freq[i])Hz)\e[0m" for i in 1:audioparams.mel_nbands],
        :mfcc in featset ? ["\e[$(color_code[:red])m$j(mfcc$i)\e[0m" for i in 1:audioparams.mfcc_ncoeffs] : String[],
        :f0 in featset ? ["\e[$(color_code[:green])m$j(f0)\e[0m"] : String[],
        "\e[$(color_code[:cyan])m$j(cntrd)\e[0m", "\e[$(color_code[:cyan])m$j(crest)\e[0m",
        "\e[$(color_code[:cyan])m$j(entrp)\e[0m", "\e[$(color_code[:cyan])m$j(flatn)\e[0m", "\e[$(color_code[:cyan])m$j(flux)\e[0m",
        "\e[$(color_code[:cyan])m$j(kurts)\e[0m", "\e[$(color_code[:cyan])m$j(rllff)\e[0m", "\e[$(color_code[:cyan])m$j(skwns)\e[0m",
        "\e[$(color_code[:cyan])m$j(decrs)\e[0m", "\e[$(color_code[:cyan])m$j(slope)\e[0m", "\e[$(color_code[:cyan])m$j(sprd)\e[0m"
    )
    for j in catch9_f
]...)

@info("Load dataset...")
d = jldopen(dsfile)
X_test = d["X_test"]
y_test = d["y_test"]
close(d)
d = jldopen(jld2file)
sole_dt = d["sole_dt"]
close(d)

interesting_rules = listrules(sole_dt,
	min_lift = 1.0,
	# min_lift = 2.0,
	min_ninstances = 0,
	min_coverage = 0.10,
	normalize = true,
    variable_names_map=variable_names
);
map(r->(consequent(r), readmetrics(r)), interesting_rules)

readmetrics.(interesting_rules)

printmodel.(
    sort(interesting_rules, by = readmetrics); 
    show_metrics = (; round_digits = 2, additional_metrics = (; height = r->SoleLogics.height(antecedent(r)))), 
    syntaxstring_kwargs = (; threshold_digits = 2), 
    variable_names_map=variable_names
);

┌ Info: Load dataset...
└ @ Main /home/paso/Documents/Aclai/audio-rules2024/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X10sZmlsZQ==.jl:53


▣ (std(skwns) < 20.92) ∧ (std(mel15=1663Hz) < 0.42) ∧ (min(mel11=1179Hz) < -4.25) ∧ (std(mel13=1407Hz) < 0.2)  ↣  Pneumonia : (ninstances = 101, ncovered = 19, coverage = 0.19, confidence = 1.0, lift = 1.91, natoms = 4, height = 3)
▣ (std(skwns) ≥ 20.92) ∧ (std(mel15=1663Hz) ≥ 0.17) ∧ (mean(mel17=1950Hz) < -4.13)  ↣  Healthy : (ninstances = 101, ncovered = 20, coverage = 0.2, confidence = 0.95, lift = 2.0, natoms = 3, height = 2)


### Pneumonia
scale: mel _htk with mfcc

In [5]:
experiment = :Pneumonia
# experiment = :Bronchiectasis
# experiment = :COPD
# experiment = :URTI
# experiment = :Bronchiolitis

# scale = :semitones
scale = :mel_htk

# featset = ()
featset = (:mfcc,)

audioparams = (
    sr = sr,
    nfft = 512,
    mel_scale = scale, # :mel_htk, :mel_slaney, :erb, :bark, :semitones, :tuned_semitones
    mel_nbands = scale == :semitones ? 14 : 26,
    mfcc_ncoeffs = scale == :semitones ? 7 : 13,
    mel_freqrange = (300, round(Int, sr / 2)),
    mel_dbscale = :mfcc in featset ? false : true,
    audio_norm = true,
)

@assert experiment in avail_exp "Unknown type of experiment: $experiment."
ds_path = "/datasets/respiratory_Healthy_" * String(experiment)
filename = "/datasets/itadata2024_" * String(experiment) * "_files"

destpath = "results/propositional/$scale"
:mfcc in featset ? destpath *= "_mfcc/" : destpath *= "/"
jld2file = destpath * "/itadata2024_" * String(experiment) * "_" * String(scale) * ".jld2"
dsfile = destpath * "/ds_test_" * String(experiment) * "_" * String(scale) * ".jld2"

d = jldopen(string((@__DIR__), ds_path, ".jld2"))
x, y = d["dataframe_validated"]
@assert x isa DataFrame
close(d)

freq = round.(Int, afe(x[1, :audio]; featset=(:get_only_freqs), audioparams...))

variable_names = vcat([
    vcat(
        ["\e[$(color_code[:yellow])m$j(mel$i=$(freq[i])Hz)\e[0m" for i in 1:audioparams.mel_nbands],
        :mfcc in featset ? ["\e[$(color_code[:red])m$j(mfcc$i)\e[0m" for i in 1:audioparams.mfcc_ncoeffs] : String[],
        :f0 in featset ? ["\e[$(color_code[:green])m$j(f0)\e[0m"] : String[],
        "\e[$(color_code[:cyan])m$j(cntrd)\e[0m", "\e[$(color_code[:cyan])m$j(crest)\e[0m",
        "\e[$(color_code[:cyan])m$j(entrp)\e[0m", "\e[$(color_code[:cyan])m$j(flatn)\e[0m", "\e[$(color_code[:cyan])m$j(flux)\e[0m",
        "\e[$(color_code[:cyan])m$j(kurts)\e[0m", "\e[$(color_code[:cyan])m$j(rllff)\e[0m", "\e[$(color_code[:cyan])m$j(skwns)\e[0m",
        "\e[$(color_code[:cyan])m$j(decrs)\e[0m", "\e[$(color_code[:cyan])m$j(slope)\e[0m", "\e[$(color_code[:cyan])m$j(sprd)\e[0m"
    )
    for j in catch9_f
]...)

@info("Load dataset...")
d = jldopen(dsfile)
X_test = d["X_test"]
y_test = d["y_test"]
close(d)
d = jldopen(jld2file)
sole_dt = d["sole_dt"]
close(d)

interesting_rules = listrules(sole_dt,
	min_lift = 1.0,
	# min_lift = 2.0,
	min_ninstances = 0,
	min_coverage = 0.10,
	normalize = true,
    variable_names_map=variable_names
);
map(r->(consequent(r), readmetrics(r)), interesting_rules)

readmetrics.(interesting_rules)

printmodel.(
    sort(interesting_rules, by = readmetrics); 
    show_metrics = (; round_digits = 2, additional_metrics = (; height = r->SoleLogics.height(antecedent(r)))), 
    syntaxstring_kwargs = (; threshold_digits = 2), 
    variable_names_map=variable_names
);

┌ Info: Load dataset...
└ @ Main /home/paso/Documents/Aclai/audio-rules2024/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X12sZmlsZQ==.jl:53


▣ (min(mfcc3) < -0.21) ∧ (min(mfcc2) ≥ 2.17) ∧ (med(mfcc7) ≥ -0.44) ∧ (mean(mfcc13) ≥ -0.08) ∧ (mean(mfcc9) < 0.2) ∧ (med(mel22=2829Hz) < 0.0)  ↣  Pneumonia : (ninstances = 101, ncovered = 18, coverage = 0.18, confidence = 0.61, lift = 1.34, natoms = 6, height = 5)
▣ (min(mfcc3) < -0.21) ∧ (min(mfcc2) < 2.17) ∧ (max(mel3=488Hz) ≥ 0.0) ∧ (med(kurts) ≥ 143.69)  ↣  Healthy : (ninstances = 101, ncovered = 27, coverage = 0.27, confidence = 0.89, lift = 1.63, natoms = 4, height = 3)
▣ (min(mfcc3) ≥ -0.21) ∧ (min(mfcc2) ≥ 1.92) ∧ (3ac(kurts) ≥ 0.01)  ↣  Pneumonia : (ninstances = 101, ncovered = 28, coverage = 0.28, confidence = 0.93, lift = 2.04, natoms = 3, height = 2)


### Bronchiectasis
scale: semitones without mfcc

In [6]:
# experiment = :Pneumonia
experiment = :Bronchiectasis
# experiment = :COPD
# experiment = :URTI
# experiment = :Bronchiolitis

scale = :semitones
# scale = :mel_htk

featset = ()
# featset = (:mfcc,)

audioparams = (
    sr = sr,
    nfft = 512,
    mel_scale = scale, # :mel_htk, :mel_slaney, :erb, :bark, :semitones, :tuned_semitones
    mel_nbands = scale == :semitones ? 14 : 26,
    mfcc_ncoeffs = scale == :semitones ? 7 : 13,
    mel_freqrange = (300, round(Int, sr / 2)),
    mel_dbscale = :mfcc in featset ? false : true,
    audio_norm = true,
)

@assert experiment in avail_exp "Unknown type of experiment: $experiment."
ds_path = "/datasets/respiratory_Healthy_" * String(experiment)
filename = "/datasets/itadata2024_" * String(experiment) * "_files"

destpath = "results/propositional/$scale"
:mfcc in featset ? destpath *= "_mfcc/" : destpath *= "/"
jld2file = destpath * "/itadata2024_" * String(experiment) * "_" * String(scale) * ".jld2"
dsfile = destpath * "/ds_test_" * String(experiment) * "_" * String(scale) * ".jld2"

d = jldopen(string((@__DIR__), ds_path, ".jld2"))
x, y = d["dataframe_validated"]
@assert x isa DataFrame
close(d)

freq = round.(Int, afe(x[1, :audio]; featset=(:get_only_freqs), audioparams...))

variable_names = vcat([
    vcat(
        ["\e[$(color_code[:yellow])m$j(mel$i=$(freq[i])Hz)\e[0m" for i in 1:audioparams.mel_nbands],
        :mfcc in featset ? ["\e[$(color_code[:red])m$j(mfcc$i)\e[0m" for i in 1:audioparams.mfcc_ncoeffs] : String[],
        :f0 in featset ? ["\e[$(color_code[:green])m$j(f0)\e[0m"] : String[],
        "\e[$(color_code[:cyan])m$j(cntrd)\e[0m", "\e[$(color_code[:cyan])m$j(crest)\e[0m",
        "\e[$(color_code[:cyan])m$j(entrp)\e[0m", "\e[$(color_code[:cyan])m$j(flatn)\e[0m", "\e[$(color_code[:cyan])m$j(flux)\e[0m",
        "\e[$(color_code[:cyan])m$j(kurts)\e[0m", "\e[$(color_code[:cyan])m$j(rllff)\e[0m", "\e[$(color_code[:cyan])m$j(skwns)\e[0m",
        "\e[$(color_code[:cyan])m$j(decrs)\e[0m", "\e[$(color_code[:cyan])m$j(slope)\e[0m", "\e[$(color_code[:cyan])m$j(sprd)\e[0m"
    )
    for j in catch9_f
]...)

@info("Load dataset...")
d = jldopen(dsfile)
X_test = d["X_test"]
y_test = d["y_test"]
close(d)
d = jldopen(jld2file)
sole_dt = d["sole_dt"]
close(d)

interesting_rules = listrules(sole_dt,
	min_lift = 1.0,
	# min_lift = 2.0,
	min_ninstances = 0,
	min_coverage = 0.10,
	normalize = true,
    variable_names_map=variable_names
);
map(r->(consequent(r), readmetrics(r)), interesting_rules)

readmetrics.(interesting_rules)

printmodel.(
    sort(interesting_rules, by = readmetrics); 
    show_metrics = (; round_digits = 2, additional_metrics = (; height = r->SoleLogics.height(antecedent(r)))), 
    syntaxstring_kwargs = (; threshold_digits = 2), 
    variable_names_map=variable_names
);

┌ Info: Load dataset...
└ @ Main /home/paso/Documents/Aclai/audio-rules2024/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X14sZmlsZQ==.jl:53


▣ std(decrs) < 2.77  ↣  Bronchiectasis : (ninstances = 36, ncovered = 18, coverage = 0.5, confidence = 1.0, lift = 2.0, natoms = 1, height = 0)
▣ std(decrs) ≥ 2.77  ↣  Healthy : (ninstances = 36, ncovered = 18, coverage = 0.5, confidence = 1.0, lift = 2.0, natoms = 1, height = 0)


### Bronchiectasis
scale: semitones with mfcc

In [7]:
# experiment = :Pneumonia
experiment = :Bronchiectasis
# experiment = :COPD
# experiment = :URTI
# experiment = :Bronchiolitis

scale = :semitones
# scale = :mel_htk

# featset = ()
featset = (:mfcc,)

audioparams = (
    sr = sr,
    nfft = 512,
    mel_scale = scale, # :mel_htk, :mel_slaney, :erb, :bark, :semitones, :tuned_semitones
    mel_nbands = scale == :semitones ? 14 : 26,
    mfcc_ncoeffs = scale == :semitones ? 7 : 13,
    mel_freqrange = (300, round(Int, sr / 2)),
    mel_dbscale = :mfcc in featset ? false : true,
    audio_norm = true,
)

@assert experiment in avail_exp "Unknown type of experiment: $experiment."
ds_path = "/datasets/respiratory_Healthy_" * String(experiment)
filename = "/datasets/itadata2024_" * String(experiment) * "_files"

destpath = "results/propositional/$scale"
:mfcc in featset ? destpath *= "_mfcc/" : destpath *= "/"
jld2file = destpath * "/itadata2024_" * String(experiment) * "_" * String(scale) * ".jld2"
dsfile = destpath * "/ds_test_" * String(experiment) * "_" * String(scale) * ".jld2"

d = jldopen(string((@__DIR__), ds_path, ".jld2"))
x, y = d["dataframe_validated"]
@assert x isa DataFrame
close(d)

freq = round.(Int, afe(x[1, :audio]; featset=(:get_only_freqs), audioparams...))

variable_names = vcat([
    vcat(
        ["\e[$(color_code[:yellow])m$j(mel$i=$(freq[i])Hz)\e[0m" for i in 1:audioparams.mel_nbands],
        :mfcc in featset ? ["\e[$(color_code[:red])m$j(mfcc$i)\e[0m" for i in 1:audioparams.mfcc_ncoeffs] : String[],
        :f0 in featset ? ["\e[$(color_code[:green])m$j(f0)\e[0m"] : String[],
        "\e[$(color_code[:cyan])m$j(cntrd)\e[0m", "\e[$(color_code[:cyan])m$j(crest)\e[0m",
        "\e[$(color_code[:cyan])m$j(entrp)\e[0m", "\e[$(color_code[:cyan])m$j(flatn)\e[0m", "\e[$(color_code[:cyan])m$j(flux)\e[0m",
        "\e[$(color_code[:cyan])m$j(kurts)\e[0m", "\e[$(color_code[:cyan])m$j(rllff)\e[0m", "\e[$(color_code[:cyan])m$j(skwns)\e[0m",
        "\e[$(color_code[:cyan])m$j(decrs)\e[0m", "\e[$(color_code[:cyan])m$j(slope)\e[0m", "\e[$(color_code[:cyan])m$j(sprd)\e[0m"
    )
    for j in catch9_f
]...)

@info("Load dataset...")
d = jldopen(dsfile)
X_test = d["X_test"]
y_test = d["y_test"]
close(d)
d = jldopen(jld2file)
sole_dt = d["sole_dt"]
close(d)

interesting_rules = listrules(sole_dt,
	min_lift = 1.0,
	# min_lift = 2.0,
	min_ninstances = 0,
	min_coverage = 0.10,
	normalize = true,
    variable_names_map=variable_names
);
map(r->(consequent(r), readmetrics(r)), interesting_rules)

readmetrics.(interesting_rules)

printmodel.(
    sort(interesting_rules, by = readmetrics); 
    show_metrics = (; round_digits = 2, additional_metrics = (; height = r->SoleLogics.height(antecedent(r)))), 
    syntaxstring_kwargs = (; threshold_digits = 2), 
    variable_names_map=variable_names
);

┌ Info: Load dataset...
└ @ Main /home/paso/Documents/Aclai/audio-rules2024/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X16sZmlsZQ==.jl:53


▣ mean(decrs) ≥ -1.49  ↣  Bronchiectasis : (ninstances = 36, ncovered = 14, coverage = 0.39, confidence = 1.0, lift = 2.57, natoms = 1, height = 0)
▣ mean(decrs) < -1.49  ↣  Healthy : (ninstances = 36, ncovered = 22, coverage = 0.61, confidence = 1.0, lift = 1.64, natoms = 1, height = 0)


### Bronchiectasis
scale: mel_htk without mfcc

In [8]:
# experiment = :Pneumonia
experiment = :Bronchiectasis
# experiment = :COPD
# experiment = :URTI
# experiment = :Bronchiolitis

# scale = :semitones
scale = :mel_htk

featset = ()
# featset = (:mfcc,)

audioparams = (
    sr = sr,
    nfft = 512,
    mel_scale = scale, # :mel_htk, :mel_slaney, :erb, :bark, :semitones, :tuned_semitones
    mel_nbands = scale == :semitones ? 14 : 26,
    mfcc_ncoeffs = scale == :semitones ? 7 : 13,
    mel_freqrange = (300, round(Int, sr / 2)),
    mel_dbscale = :mfcc in featset ? false : true,
    audio_norm = true,
)

@assert experiment in avail_exp "Unknown type of experiment: $experiment."
ds_path = "/datasets/respiratory_Healthy_" * String(experiment)
filename = "/datasets/itadata2024_" * String(experiment) * "_files"

destpath = "results/propositional/$scale"
:mfcc in featset ? destpath *= "_mfcc/" : destpath *= "/"
jld2file = destpath * "/itadata2024_" * String(experiment) * "_" * String(scale) * ".jld2"
dsfile = destpath * "/ds_test_" * String(experiment) * "_" * String(scale) * ".jld2"

d = jldopen(string((@__DIR__), ds_path, ".jld2"))
x, y = d["dataframe_validated"]
@assert x isa DataFrame
close(d)

freq = round.(Int, afe(x[1, :audio]; featset=(:get_only_freqs), audioparams...))

variable_names = vcat([
    vcat(
        ["\e[$(color_code[:yellow])m$j(mel$i=$(freq[i])Hz)\e[0m" for i in 1:audioparams.mel_nbands],
        :mfcc in featset ? ["\e[$(color_code[:red])m$j(mfcc$i)\e[0m" for i in 1:audioparams.mfcc_ncoeffs] : String[],
        :f0 in featset ? ["\e[$(color_code[:green])m$j(f0)\e[0m"] : String[],
        "\e[$(color_code[:cyan])m$j(cntrd)\e[0m", "\e[$(color_code[:cyan])m$j(crest)\e[0m",
        "\e[$(color_code[:cyan])m$j(entrp)\e[0m", "\e[$(color_code[:cyan])m$j(flatn)\e[0m", "\e[$(color_code[:cyan])m$j(flux)\e[0m",
        "\e[$(color_code[:cyan])m$j(kurts)\e[0m", "\e[$(color_code[:cyan])m$j(rllff)\e[0m", "\e[$(color_code[:cyan])m$j(skwns)\e[0m",
        "\e[$(color_code[:cyan])m$j(decrs)\e[0m", "\e[$(color_code[:cyan])m$j(slope)\e[0m", "\e[$(color_code[:cyan])m$j(sprd)\e[0m"
    )
    for j in catch9_f
]...)

@info("Load dataset...")
d = jldopen(dsfile)
X_test = d["X_test"]
y_test = d["y_test"]
close(d)
d = jldopen(jld2file)
sole_dt = d["sole_dt"]
close(d)

interesting_rules = listrules(sole_dt,
	min_lift = 1.0,
	# min_lift = 2.0,
	min_ninstances = 0,
	min_coverage = 0.10,
	normalize = true,
    variable_names_map=variable_names
);
map(r->(consequent(r), readmetrics(r)), interesting_rules)

readmetrics.(interesting_rules)

printmodel.(
    sort(interesting_rules, by = readmetrics); 
    show_metrics = (; round_digits = 2, additional_metrics = (; height = r->SoleLogics.height(antecedent(r)))), 
    syntaxstring_kwargs = (; threshold_digits = 2), 
    variable_names_map=variable_names
);

┌ Info: Load dataset...
└ @ Main /home/paso/Documents/Aclai/audio-rules2024/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X21sZmlsZQ==.jl:53


▣ med(decrs) < -0.75  ↣  Healthy : (ninstances = 36, ncovered = 16, coverage = 0.44, confidence = 1.0, lift = 2.12, natoms = 1, height = 0)
▣ med(decrs) ≥ -0.75  ↣  Bronchiectasis : (ninstances = 36, ncovered = 20, coverage = 0.56, confidence = 0.95, lift = 1.8, natoms = 1, height = 0)


### Bronchiectasis
scale: mel_htk with mfcc

In [9]:
# experiment = :Pneumonia
experiment = :Bronchiectasis
# experiment = :COPD
# experiment = :URTI
# experiment = :Bronchiolitis

# scale = :semitones
scale = :mel_htk

# featset = ()
featset = (:mfcc,)

audioparams = (
    sr = sr,
    nfft = 512,
    mel_scale = scale, # :mel_htk, :mel_slaney, :erb, :bark, :semitones, :tuned_semitones
    mel_nbands = scale == :semitones ? 14 : 26,
    mfcc_ncoeffs = scale == :semitones ? 7 : 13,
    mel_freqrange = (300, round(Int, sr / 2)),
    mel_dbscale = :mfcc in featset ? false : true,
    audio_norm = true,
)

@assert experiment in avail_exp "Unknown type of experiment: $experiment."
ds_path = "/datasets/respiratory_Healthy_" * String(experiment)
filename = "/datasets/itadata2024_" * String(experiment) * "_files"

destpath = "results/propositional/$scale"
:mfcc in featset ? destpath *= "_mfcc/" : destpath *= "/"
jld2file = destpath * "/itadata2024_" * String(experiment) * "_" * String(scale) * ".jld2"
dsfile = destpath * "/ds_test_" * String(experiment) * "_" * String(scale) * ".jld2"

d = jldopen(string((@__DIR__), ds_path, ".jld2"))
x, y = d["dataframe_validated"]
@assert x isa DataFrame
close(d)

freq = round.(Int, afe(x[1, :audio]; featset=(:get_only_freqs), audioparams...))

variable_names = vcat([
    vcat(
        ["\e[$(color_code[:yellow])m$j(mel$i=$(freq[i])Hz)\e[0m" for i in 1:audioparams.mel_nbands],
        :mfcc in featset ? ["\e[$(color_code[:red])m$j(mfcc$i)\e[0m" for i in 1:audioparams.mfcc_ncoeffs] : String[],
        :f0 in featset ? ["\e[$(color_code[:green])m$j(f0)\e[0m"] : String[],
        "\e[$(color_code[:cyan])m$j(cntrd)\e[0m", "\e[$(color_code[:cyan])m$j(crest)\e[0m",
        "\e[$(color_code[:cyan])m$j(entrp)\e[0m", "\e[$(color_code[:cyan])m$j(flatn)\e[0m", "\e[$(color_code[:cyan])m$j(flux)\e[0m",
        "\e[$(color_code[:cyan])m$j(kurts)\e[0m", "\e[$(color_code[:cyan])m$j(rllff)\e[0m", "\e[$(color_code[:cyan])m$j(skwns)\e[0m",
        "\e[$(color_code[:cyan])m$j(decrs)\e[0m", "\e[$(color_code[:cyan])m$j(slope)\e[0m", "\e[$(color_code[:cyan])m$j(sprd)\e[0m"
    )
    for j in catch9_f
]...)

@info("Load dataset...")
d = jldopen(dsfile)
X_test = d["X_test"]
y_test = d["y_test"]
close(d)
d = jldopen(jld2file)
sole_dt = d["sole_dt"]
close(d)

interesting_rules = listrules(sole_dt,
	min_lift = 1.0,
	# min_lift = 2.0,
	min_ninstances = 0,
	min_coverage = 0.10,
	normalize = true,
    variable_names_map=variable_names
);
map(r->(consequent(r), readmetrics(r)), interesting_rules)

readmetrics.(interesting_rules)

printmodel.(
    sort(interesting_rules, by = readmetrics); 
    show_metrics = (; round_digits = 2, additional_metrics = (; height = r->SoleLogics.height(antecedent(r)))), 
    syntaxstring_kwargs = (; threshold_digits = 2), 
    variable_names_map=variable_names
);

┌ Info: Load dataset...
└ @ Main /home/paso/Documents/Aclai/audio-rules2024/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X23sZmlsZQ==.jl:53


▣ mean(crest) < 115.92  ↣  Bronchiectasis : (ninstances = 36, ncovered = 18, coverage = 0.5, confidence = 1.0, lift = 2.0, natoms = 1, height = 0)
▣ mean(crest) ≥ 115.92  ↣  Healthy : (ninstances = 36, ncovered = 18, coverage = 0.5, confidence = 1.0, lift = 2.0, natoms = 1, height = 0)


### COPD
scale: semitones without mfcc

In [10]:
# experiment = :Pneumonia
# experiment = :Bronchiectasis
experiment = :COPD
# experiment = :URTI
# experiment = :Bronchiolitis

scale = :semitones
# scale = :mel_htk

featset = ()
# featset = (:mfcc,)

audioparams = (
    sr = sr,
    nfft = 512,
    mel_scale = scale, # :mel_htk, :mel_slaney, :erb, :bark, :semitones, :tuned_semitones
    mel_nbands = scale == :semitones ? 14 : 26,
    mfcc_ncoeffs = scale == :semitones ? 7 : 13,
    mel_freqrange = (300, round(Int, sr / 2)),
    mel_dbscale = :mfcc in featset ? false : true,
    audio_norm = true,
)

@assert experiment in avail_exp "Unknown type of experiment: $experiment."
ds_path = "/datasets/respiratory_Healthy_" * String(experiment)
filename = "/datasets/itadata2024_" * String(experiment) * "_files"

destpath = "results/propositional/$scale"
:mfcc in featset ? destpath *= "_mfcc/" : destpath *= "/"
jld2file = destpath * "/itadata2024_" * String(experiment) * "_" * String(scale) * ".jld2"
dsfile = destpath * "/ds_test_" * String(experiment) * "_" * String(scale) * ".jld2"

d = jldopen(string((@__DIR__), ds_path, ".jld2"))
x, y = d["dataframe_validated"]
@assert x isa DataFrame
close(d)

freq = round.(Int, afe(x[1, :audio]; featset=(:get_only_freqs), audioparams...))

variable_names = vcat([
    vcat(
        ["\e[$(color_code[:yellow])m$j(mel$i=$(freq[i])Hz)\e[0m" for i in 1:audioparams.mel_nbands],
        :mfcc in featset ? ["\e[$(color_code[:red])m$j(mfcc$i)\e[0m" for i in 1:audioparams.mfcc_ncoeffs] : String[],
        :f0 in featset ? ["\e[$(color_code[:green])m$j(f0)\e[0m"] : String[],
        "\e[$(color_code[:cyan])m$j(cntrd)\e[0m", "\e[$(color_code[:cyan])m$j(crest)\e[0m",
        "\e[$(color_code[:cyan])m$j(entrp)\e[0m", "\e[$(color_code[:cyan])m$j(flatn)\e[0m", "\e[$(color_code[:cyan])m$j(flux)\e[0m",
        "\e[$(color_code[:cyan])m$j(kurts)\e[0m", "\e[$(color_code[:cyan])m$j(rllff)\e[0m", "\e[$(color_code[:cyan])m$j(skwns)\e[0m",
        "\e[$(color_code[:cyan])m$j(decrs)\e[0m", "\e[$(color_code[:cyan])m$j(slope)\e[0m", "\e[$(color_code[:cyan])m$j(sprd)\e[0m"
    )
    for j in catch9_f
]...)

@info("Load dataset...")
d = jldopen(dsfile)
X_test = d["X_test"]
y_test = d["y_test"]
close(d)
d = jldopen(jld2file)
sole_dt = d["sole_dt"]
close(d)

interesting_rules = listrules(sole_dt,
	min_lift = 1.0,
	# min_lift = 2.0,
	min_ninstances = 0,
	min_coverage = 0.10,
	normalize = true,
    variable_names_map=variable_names
);
map(r->(consequent(r), readmetrics(r)), interesting_rules)

readmetrics.(interesting_rules)

printmodel.(
    sort(interesting_rules, by = readmetrics); 
    show_metrics = (; round_digits = 2, additional_metrics = (; height = r->SoleLogics.height(antecedent(r)))), 
    syntaxstring_kwargs = (; threshold_digits = 2), 
    variable_names_map=variable_names
);

┌ Info: Load dataset...
└ @ Main /home/paso/Documents/Aclai/audio-rules2024/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X25sZmlsZQ==.jl:53


▣ (med(entrp) < 0.21) ∧ (min(mel12=2383Hz) ≥ -8.02) ∧ (std(mel10=1687Hz) ≥ 0.15) ∧ (mean(slope) ≥ -0.03)  ↣  Healthy : (ninstances = 110, ncovered = 26, coverage = 0.24, confidence = 0.96, lift = 1.86, natoms = 4, height = 3)
▣ (med(entrp) ≥ 0.23) ∧ (min(mel14=3366Hz) ≥ -5.87)  ↣  COPD : (ninstances = 110, ncovered = 30, coverage = 0.27, confidence = 0.97, lift = 2.01, natoms = 2, height = 1)


### COPD
scale: semitones with mfcc

In [11]:
# experiment = :Pneumonia
# experiment = :Bronchiectasis
experiment = :COPD
# experiment = :URTI
# experiment = :Bronchiolitis

scale = :semitones
# scale = :mel_htk

# featset = ()
featset = (:mfcc,)

audioparams = (
    sr = sr,
    nfft = 512,
    mel_scale = scale, # :mel_htk, :mel_slaney, :erb, :bark, :semitones, :tuned_semitones
    mel_nbands = scale == :semitones ? 14 : 26,
    mfcc_ncoeffs = scale == :semitones ? 7 : 13,
    mel_freqrange = (300, round(Int, sr / 2)),
    mel_dbscale = :mfcc in featset ? false : true,
    audio_norm = true,
)

@assert experiment in avail_exp "Unknown type of experiment: $experiment."
ds_path = "/datasets/respiratory_Healthy_" * String(experiment)
filename = "/datasets/itadata2024_" * String(experiment) * "_files"

destpath = "results/propositional/$scale"
:mfcc in featset ? destpath *= "_mfcc/" : destpath *= "/"
jld2file = destpath * "/itadata2024_" * String(experiment) * "_" * String(scale) * ".jld2"
dsfile = destpath * "/ds_test_" * String(experiment) * "_" * String(scale) * ".jld2"

d = jldopen(string((@__DIR__), ds_path, ".jld2"))
x, y = d["dataframe_validated"]
@assert x isa DataFrame
close(d)

freq = round.(Int, afe(x[1, :audio]; featset=(:get_only_freqs), audioparams...))

variable_names = vcat([
    vcat(
        ["\e[$(color_code[:yellow])m$j(mel$i=$(freq[i])Hz)\e[0m" for i in 1:audioparams.mel_nbands],
        :mfcc in featset ? ["\e[$(color_code[:red])m$j(mfcc$i)\e[0m" for i in 1:audioparams.mfcc_ncoeffs] : String[],
        :f0 in featset ? ["\e[$(color_code[:green])m$j(f0)\e[0m"] : String[],
        "\e[$(color_code[:cyan])m$j(cntrd)\e[0m", "\e[$(color_code[:cyan])m$j(crest)\e[0m",
        "\e[$(color_code[:cyan])m$j(entrp)\e[0m", "\e[$(color_code[:cyan])m$j(flatn)\e[0m", "\e[$(color_code[:cyan])m$j(flux)\e[0m",
        "\e[$(color_code[:cyan])m$j(kurts)\e[0m", "\e[$(color_code[:cyan])m$j(rllff)\e[0m", "\e[$(color_code[:cyan])m$j(skwns)\e[0m",
        "\e[$(color_code[:cyan])m$j(decrs)\e[0m", "\e[$(color_code[:cyan])m$j(slope)\e[0m", "\e[$(color_code[:cyan])m$j(sprd)\e[0m"
    )
    for j in catch9_f
]...)

@info("Load dataset...")
d = jldopen(dsfile)
X_test = d["X_test"]
y_test = d["y_test"]
close(d)
d = jldopen(jld2file)
sole_dt = d["sole_dt"]
close(d)

interesting_rules = listrules(sole_dt,
	min_lift = 1.0,
	# min_lift = 2.0,
	min_ninstances = 0,
	min_coverage = 0.10,
	normalize = true,
    variable_names_map=variable_names
);
map(r->(consequent(r), readmetrics(r)), interesting_rules)

readmetrics.(interesting_rules)

printmodel.(
    sort(interesting_rules, by = readmetrics); 
    show_metrics = (; round_digits = 2, additional_metrics = (; height = r->SoleLogics.height(antecedent(r)))), 
    syntaxstring_kwargs = (; threshold_digits = 2), 
    variable_names_map=variable_names
);

┌ Info: Load dataset...
└ @ Main /home/paso/Documents/Aclai/audio-rules2024/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X30sZmlsZQ==.jl:53


▣ (mean(crest) ≥ 120.66) ∧ (min(mfcc2) ≥ 1.55) ∧ (med(slope) < -0.02) ∧ (max(mfcc5) < 0.83) ∧ (max(mfcc6) ≥ 0.02)  ↣  COPD : (ninstances = 110, ncovered = 18, coverage = 0.16, confidence = 0.89, lift = 1.55, natoms = 5, height = 4)
▣ (mean(crest) < 120.66) ∧ (qnt(mel6=845Hz) ≥ 1.73)  ↣  COPD : (ninstances = 110, ncovered = 32, coverage = 0.29, confidence = 0.94, lift = 1.64, natoms = 2, height = 1)
▣ (mean(crest) ≥ 120.66) ∧ (min(mfcc2) < 1.55) ∧ (med(mel13=2832Hz) < 0.0) ∧ (mean(slope) ≥ -0.05)  ↣  Healthy : (ninstances = 110, ncovered = 35, coverage = 0.32, confidence = 0.97, lift = 2.27, natoms = 4, height = 3)


### COPD
scale: mel_htk without mfcc

In [12]:
# experiment = :Pneumonia
# experiment = :Bronchiectasis
experiment = :COPD
# experiment = :URTI
# experiment = :Bronchiolitis

# scale = :semitones
scale = :mel_htk

featset = ()
# featset = (:mfcc,)

audioparams = (
    sr = sr,
    nfft = 512,
    mel_scale = scale, # :mel_htk, :mel_slaney, :erb, :bark, :semitones, :tuned_semitones
    mel_nbands = scale == :semitones ? 14 : 26,
    mfcc_ncoeffs = scale == :semitones ? 7 : 13,
    mel_freqrange = (300, round(Int, sr / 2)),
    mel_dbscale = :mfcc in featset ? false : true,
    audio_norm = true,
)

@assert experiment in avail_exp "Unknown type of experiment: $experiment."
ds_path = "/datasets/respiratory_Healthy_" * String(experiment)
filename = "/datasets/itadata2024_" * String(experiment) * "_files"

destpath = "results/propositional/$scale"
:mfcc in featset ? destpath *= "_mfcc/" : destpath *= "/"
jld2file = destpath * "/itadata2024_" * String(experiment) * "_" * String(scale) * ".jld2"
dsfile = destpath * "/ds_test_" * String(experiment) * "_" * String(scale) * ".jld2"

d = jldopen(string((@__DIR__), ds_path, ".jld2"))
x, y = d["dataframe_validated"]
@assert x isa DataFrame
close(d)

freq = round.(Int, afe(x[1, :audio]; featset=(:get_only_freqs), audioparams...))

variable_names = vcat([
    vcat(
        ["\e[$(color_code[:yellow])m$j(mel$i=$(freq[i])Hz)\e[0m" for i in 1:audioparams.mel_nbands],
        :mfcc in featset ? ["\e[$(color_code[:red])m$j(mfcc$i)\e[0m" for i in 1:audioparams.mfcc_ncoeffs] : String[],
        :f0 in featset ? ["\e[$(color_code[:green])m$j(f0)\e[0m"] : String[],
        "\e[$(color_code[:cyan])m$j(cntrd)\e[0m", "\e[$(color_code[:cyan])m$j(crest)\e[0m",
        "\e[$(color_code[:cyan])m$j(entrp)\e[0m", "\e[$(color_code[:cyan])m$j(flatn)\e[0m", "\e[$(color_code[:cyan])m$j(flux)\e[0m",
        "\e[$(color_code[:cyan])m$j(kurts)\e[0m", "\e[$(color_code[:cyan])m$j(rllff)\e[0m", "\e[$(color_code[:cyan])m$j(skwns)\e[0m",
        "\e[$(color_code[:cyan])m$j(decrs)\e[0m", "\e[$(color_code[:cyan])m$j(slope)\e[0m", "\e[$(color_code[:cyan])m$j(sprd)\e[0m"
    )
    for j in catch9_f
]...)

@info("Load dataset...")
d = jldopen(dsfile)
X_test = d["X_test"]
y_test = d["y_test"]
close(d)
d = jldopen(jld2file)
sole_dt = d["sole_dt"]
close(d)

interesting_rules = listrules(sole_dt,
	min_lift = 1.0,
	# min_lift = 2.0,
	min_ninstances = 0,
	min_coverage = 0.10,
	normalize = true,
    variable_names_map=variable_names
);
map(r->(consequent(r), readmetrics(r)), interesting_rules)

readmetrics.(interesting_rules)

printmodel.(
    sort(interesting_rules, by = readmetrics); 
    show_metrics = (; round_digits = 2, additional_metrics = (; height = r->SoleLogics.height(antecedent(r)))), 
    syntaxstring_kwargs = (; threshold_digits = 2), 
    variable_names_map=variable_names
);

┌ Info: Load dataset...
└ @ Main /home/paso/Documents/Aclai/audio-rules2024/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X32sZmlsZQ==.jl:53


▣ (med(entrp) < 0.21) ∧ (min(mel23=3037Hz) < -8.13)  ↣  COPD : (ninstances = 110, ncovered = 11, coverage = 0.1, confidence = 1.0, lift = 1.9, natoms = 2, height = 1)
▣ (med(entrp) ≥ 0.21) ∧ (max(rllff) < 601.56) ∧ (med(cntrd) ≥ 22.28)  ↣  COPD : (ninstances = 110, ncovered = 33, coverage = 0.3, confidence = 0.97, lift = 1.84, natoms = 3, height = 2)
▣ (med(entrp) < 0.21) ∧ (min(mel23=3037Hz) ≥ -8.13) ∧ (min(mel2=421Hz) < -4.04) ∧ (std(mel13=1407Hz) ≥ 0.19) ∧ (max(mel3=488Hz) < 0.02)  ↣  Healthy : (ninstances = 110, ncovered = 39, coverage = 0.35, confidence = 0.97, lift = 2.06, natoms = 5, height = 4)


### COPD
scale: mel_htk with mfcc

In [13]:
# experiment = :Pneumonia
# experiment = :Bronchiectasis
experiment = :COPD
# experiment = :URTI
# experiment = :Bronchiolitis

# scale = :semitones
scale = :mel_htk

# featset = ()
featset = (:mfcc,)

audioparams = (
    sr = sr,
    nfft = 512,
    mel_scale = scale, # :mel_htk, :mel_slaney, :erb, :bark, :semitones, :tuned_semitones
    mel_nbands = scale == :semitones ? 14 : 26,
    mfcc_ncoeffs = scale == :semitones ? 7 : 13,
    mel_freqrange = (300, round(Int, sr / 2)),
    mel_dbscale = :mfcc in featset ? false : true,
    audio_norm = true,
)

@assert experiment in avail_exp "Unknown type of experiment: $experiment."
ds_path = "/datasets/respiratory_Healthy_" * String(experiment)
filename = "/datasets/itadata2024_" * String(experiment) * "_files"

destpath = "results/propositional/$scale"
:mfcc in featset ? destpath *= "_mfcc/" : destpath *= "/"
jld2file = destpath * "/itadata2024_" * String(experiment) * "_" * String(scale) * ".jld2"
dsfile = destpath * "/ds_test_" * String(experiment) * "_" * String(scale) * ".jld2"

d = jldopen(string((@__DIR__), ds_path, ".jld2"))
x, y = d["dataframe_validated"]
@assert x isa DataFrame
close(d)

freq = round.(Int, afe(x[1, :audio]; featset=(:get_only_freqs), audioparams...))

variable_names = vcat([
    vcat(
        ["\e[$(color_code[:yellow])m$j(mel$i=$(freq[i])Hz)\e[0m" for i in 1:audioparams.mel_nbands],
        :mfcc in featset ? ["\e[$(color_code[:red])m$j(mfcc$i)\e[0m" for i in 1:audioparams.mfcc_ncoeffs] : String[],
        :f0 in featset ? ["\e[$(color_code[:green])m$j(f0)\e[0m"] : String[],
        "\e[$(color_code[:cyan])m$j(cntrd)\e[0m", "\e[$(color_code[:cyan])m$j(crest)\e[0m",
        "\e[$(color_code[:cyan])m$j(entrp)\e[0m", "\e[$(color_code[:cyan])m$j(flatn)\e[0m", "\e[$(color_code[:cyan])m$j(flux)\e[0m",
        "\e[$(color_code[:cyan])m$j(kurts)\e[0m", "\e[$(color_code[:cyan])m$j(rllff)\e[0m", "\e[$(color_code[:cyan])m$j(skwns)\e[0m",
        "\e[$(color_code[:cyan])m$j(decrs)\e[0m", "\e[$(color_code[:cyan])m$j(slope)\e[0m", "\e[$(color_code[:cyan])m$j(sprd)\e[0m"
    )
    for j in catch9_f
]...)

@info("Load dataset...")
d = jldopen(dsfile)
X_test = d["X_test"]
y_test = d["y_test"]
close(d)
d = jldopen(jld2file)
sole_dt = d["sole_dt"]
close(d)

interesting_rules = listrules(sole_dt,
	min_lift = 1.0,
	# min_lift = 2.0,
	min_ninstances = 0,
	min_coverage = 0.10,
	normalize = true,
    variable_names_map=variable_names
);
map(r->(consequent(r), readmetrics(r)), interesting_rules)

readmetrics.(interesting_rules)

printmodel.(
    sort(interesting_rules, by = readmetrics); 
    show_metrics = (; round_digits = 2, additional_metrics = (; height = r->SoleLogics.height(antecedent(r)))), 
    syntaxstring_kwargs = (; threshold_digits = 2), 
    variable_names_map=variable_names
);

┌ Info: Load dataset...
└ @ Main /home/paso/Documents/Aclai/audio-rules2024/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X34sZmlsZQ==.jl:53


▣ (med(entrp) < 0.21) ∧ (min(mel26=3738Hz) ≥ 0.0) ∧ (min(mfcc3) < 0.0) ∧ (min(mel2=421Hz) ≥ 0.0) ∧ (mean(slope) ≥ -0.04)  ↣  Healthy : (ninstances = 110, ncovered = 11, coverage = 0.1, confidence = 0.91, lift = 1.72, natoms = 5, height = 4)
▣ (med(entrp) < 0.21) ∧ (min(mel26=3738Hz) ≥ 0.0) ∧ (min(mfcc3) < 0.0) ∧ (min(mel2=421Hz) < 0.0)  ↣  Healthy : (ninstances = 110, ncovered = 36, coverage = 0.33, confidence = 0.92, lift = 1.74, natoms = 4, height = 3)
▣ (med(entrp) ≥ 0.21) ∧ (min(mfcc3) ≥ -1.83) ∧ (qnt(mel2=421Hz) ≥ 1.76)  ↣  COPD : (ninstances = 110, ncovered = 38, coverage = 0.35, confidence = 0.87, lift = 1.84, natoms = 3, height = 2)


### URTI
scale: semitones without mfcc

In [14]:
# experiment = :Pneumonia
# experiment = :Bronchiectasis
# experiment = :COPD
experiment = :URTI
# experiment = :Bronchiolitis

scale = :semitones
# scale = :mel_htk

featset = ()
# featset = (:mfcc,)

audioparams = (
    sr = sr,
    nfft = 512,
    mel_scale = scale, # :mel_htk, :mel_slaney, :erb, :bark, :semitones, :tuned_semitones
    mel_nbands = scale == :semitones ? 14 : 26,
    mfcc_ncoeffs = scale == :semitones ? 7 : 13,
    mel_freqrange = (300, round(Int, sr / 2)),
    mel_dbscale = :mfcc in featset ? false : true,
    audio_norm = true,
)

@assert experiment in avail_exp "Unknown type of experiment: $experiment."
ds_path = "/datasets/respiratory_Healthy_" * String(experiment)
filename = "/datasets/itadata2024_" * String(experiment) * "_files"

destpath = "results/propositional/$scale"
:mfcc in featset ? destpath *= "_mfcc/" : destpath *= "/"
jld2file = destpath * "/itadata2024_" * String(experiment) * "_" * String(scale) * ".jld2"
dsfile = destpath * "/ds_test_" * String(experiment) * "_" * String(scale) * ".jld2"

d = jldopen(string((@__DIR__), ds_path, ".jld2"))
x, y = d["dataframe_validated"]
@assert x isa DataFrame
close(d)

freq = round.(Int, afe(x[1, :audio]; featset=(:get_only_freqs), audioparams...))

variable_names = vcat([
    vcat(
        ["\e[$(color_code[:yellow])m$j(mel$i=$(freq[i])Hz)\e[0m" for i in 1:audioparams.mel_nbands],
        :mfcc in featset ? ["\e[$(color_code[:red])m$j(mfcc$i)\e[0m" for i in 1:audioparams.mfcc_ncoeffs] : String[],
        :f0 in featset ? ["\e[$(color_code[:green])m$j(f0)\e[0m"] : String[],
        "\e[$(color_code[:cyan])m$j(cntrd)\e[0m", "\e[$(color_code[:cyan])m$j(crest)\e[0m",
        "\e[$(color_code[:cyan])m$j(entrp)\e[0m", "\e[$(color_code[:cyan])m$j(flatn)\e[0m", "\e[$(color_code[:cyan])m$j(flux)\e[0m",
        "\e[$(color_code[:cyan])m$j(kurts)\e[0m", "\e[$(color_code[:cyan])m$j(rllff)\e[0m", "\e[$(color_code[:cyan])m$j(skwns)\e[0m",
        "\e[$(color_code[:cyan])m$j(decrs)\e[0m", "\e[$(color_code[:cyan])m$j(slope)\e[0m", "\e[$(color_code[:cyan])m$j(sprd)\e[0m"
    )
    for j in catch9_f
]...)

@info("Load dataset...")
d = jldopen(dsfile)
X_test = d["X_test"]
y_test = d["y_test"]
close(d)
d = jldopen(jld2file)
sole_dt = d["sole_dt"]
close(d)

interesting_rules = listrules(sole_dt,
	min_lift = 1.0,
	# min_lift = 2.0,
	min_ninstances = 0,
	min_coverage = 0.10,
	normalize = true,
    variable_names_map=variable_names
);
map(r->(consequent(r), readmetrics(r)), interesting_rules)

readmetrics.(interesting_rules)

printmodel.(
    sort(interesting_rules, by = readmetrics); 
    show_metrics = (; round_digits = 2, additional_metrics = (; height = r->SoleLogics.height(antecedent(r)))), 
    syntaxstring_kwargs = (; threshold_digits = 2), 
    variable_names_map=variable_names
);

┌ Info: Load dataset...
└ @ Main /home/paso/Documents/Aclai/audio-rules2024/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X36sZmlsZQ==.jl:53


▣ (mean(skwns) ≥ 17.49) ∧ (min(mel1=357Hz) < -3.17) ∧ (mean(slope) < -0.02) ∧ (qnt(cntrd) < 2.05)  ↣  Healthy : (ninstances = 86, ncovered = 10, coverage = 0.12, confidence = 0.6, lift = 1.12, natoms = 4, height = 3)
▣ (mean(skwns) ≥ 17.49) ∧ (min(mel1=357Hz) < -3.17) ∧ (mean(slope) < -0.02) ∧ (qnt(cntrd) ≥ 2.05) ∧ (max(mel10=1687Hz) ≥ -4.21) ∧ (std(decrs) < 8.13) ∧ (max(flatn) ≥ 0.0) ∧ (max(slope) < -0.0)  ↣  Healthy : (ninstances = 86, ncovered = 10, coverage = 0.12, confidence = 0.7, lift = 1.31, natoms = 8, height = 7)
▣ (mean(skwns) < 17.49) ∧ (mean(mel4=599Hz) ≥ -5.19) ∧ (std(mel1=357Hz) ≥ 0.48) ∧ (3ac(mel3=504Hz) < 0.15) ∧ (qnt(mel14=3366Hz) ≥ 2.1)  ↣  URTI : (ninstances = 86, ncovered = 10, coverage = 0.12, confidence = 0.9, lift = 1.94, natoms = 5, height = 4)


### URTI
scale: semitones with mfcc

In [15]:
# experiment = :Pneumonia
# experiment = :Bronchiectasis
# experiment = :COPD
experiment = :URTI
# experiment = :Bronchiolitis

scale = :semitones
# scale = :mel_htk

# featset = ()
featset = (:mfcc,)

audioparams = (
    sr = sr,
    nfft = 512,
    mel_scale = scale, # :mel_htk, :mel_slaney, :erb, :bark, :semitones, :tuned_semitones
    mel_nbands = scale == :semitones ? 14 : 26,
    mfcc_ncoeffs = scale == :semitones ? 7 : 13,
    mel_freqrange = (300, round(Int, sr / 2)),
    mel_dbscale = :mfcc in featset ? false : true,
    audio_norm = true,
)

@assert experiment in avail_exp "Unknown type of experiment: $experiment."
ds_path = "/datasets/respiratory_Healthy_" * String(experiment)
filename = "/datasets/itadata2024_" * String(experiment) * "_files"

destpath = "results/propositional/$scale"
:mfcc in featset ? destpath *= "_mfcc/" : destpath *= "/"
jld2file = destpath * "/itadata2024_" * String(experiment) * "_" * String(scale) * ".jld2"
dsfile = destpath * "/ds_test_" * String(experiment) * "_" * String(scale) * ".jld2"

d = jldopen(string((@__DIR__), ds_path, ".jld2"))
x, y = d["dataframe_validated"]
@assert x isa DataFrame
close(d)

freq = round.(Int, afe(x[1, :audio]; featset=(:get_only_freqs), audioparams...))

variable_names = vcat([
    vcat(
        ["\e[$(color_code[:yellow])m$j(mel$i=$(freq[i])Hz)\e[0m" for i in 1:audioparams.mel_nbands],
        :mfcc in featset ? ["\e[$(color_code[:red])m$j(mfcc$i)\e[0m" for i in 1:audioparams.mfcc_ncoeffs] : String[],
        :f0 in featset ? ["\e[$(color_code[:green])m$j(f0)\e[0m"] : String[],
        "\e[$(color_code[:cyan])m$j(cntrd)\e[0m", "\e[$(color_code[:cyan])m$j(crest)\e[0m",
        "\e[$(color_code[:cyan])m$j(entrp)\e[0m", "\e[$(color_code[:cyan])m$j(flatn)\e[0m", "\e[$(color_code[:cyan])m$j(flux)\e[0m",
        "\e[$(color_code[:cyan])m$j(kurts)\e[0m", "\e[$(color_code[:cyan])m$j(rllff)\e[0m", "\e[$(color_code[:cyan])m$j(skwns)\e[0m",
        "\e[$(color_code[:cyan])m$j(decrs)\e[0m", "\e[$(color_code[:cyan])m$j(slope)\e[0m", "\e[$(color_code[:cyan])m$j(sprd)\e[0m"
    )
    for j in catch9_f
]...)

@info("Load dataset...")
d = jldopen(dsfile)
X_test = d["X_test"]
y_test = d["y_test"]
close(d)
d = jldopen(jld2file)
sole_dt = d["sole_dt"]
close(d)

interesting_rules = listrules(sole_dt,
	min_lift = 1.0,
	# min_lift = 2.0,
	min_ninstances = 0,
	min_coverage = 0.10,
	normalize = true,
    variable_names_map=variable_names
);
map(r->(consequent(r), readmetrics(r)), interesting_rules)

readmetrics.(interesting_rules)

printmodel.(
    sort(interesting_rules, by = readmetrics); 
    show_metrics = (; round_digits = 2, additional_metrics = (; height = r->SoleLogics.height(antecedent(r)))), 
    syntaxstring_kwargs = (; threshold_digits = 2), 
    variable_names_map=variable_names
);

┌ Info: Load dataset...
└ @ Main /home/paso/Documents/Aclai/audio-rules2024/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X41sZmlsZQ==.jl:53


▣ (mean(mfcc2) < 3.53) ∧ (med(mfcc2) ≥ 1.49) ∧ (max(mfcc3) ≥ 0.26) ∧ (qnt(kurts) < 2.14) ∧ (bsm(mfcc1) ≥ 10.5) ∧ (bsd(mfcc6) < 4.5)  ↣  Healthy : (ninstances = 86, ncovered = 9, coverage = 0.1, confidence = 0.78, lift = 1.49, natoms = 6, height = 5)
▣ (mean(mfcc2) ≥ 3.53) ∧ (mean(mel3=504Hz) ≥ 0.0) ∧ (max(mel13=2832Hz) < 0.0)  ↣  URTI : (ninstances = 86, ncovered = 14, coverage = 0.16, confidence = 0.86, lift = 1.8, natoms = 3, height = 2)


### URTI
scale: mel_htk without mfcc

In [16]:
# experiment = :Pneumonia
# experiment = :Bronchiectasis
# experiment = :COPD
experiment = :URTI
# experiment = :Bronchiolitis

# scale = :semitones
scale = :mel_htk

featset = ()
# featset = (:mfcc,)

audioparams = (
    sr = sr,
    nfft = 512,
    mel_scale = scale, # :mel_htk, :mel_slaney, :erb, :bark, :semitones, :tuned_semitones
    mel_nbands = scale == :semitones ? 14 : 26,
    mfcc_ncoeffs = scale == :semitones ? 7 : 13,
    mel_freqrange = (300, round(Int, sr / 2)),
    mel_dbscale = :mfcc in featset ? false : true,
    audio_norm = true,
)

@assert experiment in avail_exp "Unknown type of experiment: $experiment."
ds_path = "/datasets/respiratory_Healthy_" * String(experiment)
filename = "/datasets/itadata2024_" * String(experiment) * "_files"

destpath = "results/propositional/$scale"
:mfcc in featset ? destpath *= "_mfcc/" : destpath *= "/"
jld2file = destpath * "/itadata2024_" * String(experiment) * "_" * String(scale) * ".jld2"
dsfile = destpath * "/ds_test_" * String(experiment) * "_" * String(scale) * ".jld2"

d = jldopen(string((@__DIR__), ds_path, ".jld2"))
x, y = d["dataframe_validated"]
@assert x isa DataFrame
close(d)

freq = round.(Int, afe(x[1, :audio]; featset=(:get_only_freqs), audioparams...))

variable_names = vcat([
    vcat(
        ["\e[$(color_code[:yellow])m$j(mel$i=$(freq[i])Hz)\e[0m" for i in 1:audioparams.mel_nbands],
        :mfcc in featset ? ["\e[$(color_code[:red])m$j(mfcc$i)\e[0m" for i in 1:audioparams.mfcc_ncoeffs] : String[],
        :f0 in featset ? ["\e[$(color_code[:green])m$j(f0)\e[0m"] : String[],
        "\e[$(color_code[:cyan])m$j(cntrd)\e[0m", "\e[$(color_code[:cyan])m$j(crest)\e[0m",
        "\e[$(color_code[:cyan])m$j(entrp)\e[0m", "\e[$(color_code[:cyan])m$j(flatn)\e[0m", "\e[$(color_code[:cyan])m$j(flux)\e[0m",
        "\e[$(color_code[:cyan])m$j(kurts)\e[0m", "\e[$(color_code[:cyan])m$j(rllff)\e[0m", "\e[$(color_code[:cyan])m$j(skwns)\e[0m",
        "\e[$(color_code[:cyan])m$j(decrs)\e[0m", "\e[$(color_code[:cyan])m$j(slope)\e[0m", "\e[$(color_code[:cyan])m$j(sprd)\e[0m"
    )
    for j in catch9_f
]...)

@info("Load dataset...")
d = jldopen(dsfile)
X_test = d["X_test"]
y_test = d["y_test"]
close(d)
d = jldopen(jld2file)
sole_dt = d["sole_dt"]
close(d)

interesting_rules = listrules(sole_dt,
	min_lift = 1.0,
	# min_lift = 2.0,
	min_ninstances = 0,
	min_coverage = 0.10,
	normalize = true,
    variable_names_map=variable_names
);
map(r->(consequent(r), readmetrics(r)), interesting_rules)

readmetrics.(interesting_rules)

printmodel.(
    sort(interesting_rules, by = readmetrics); 
    show_metrics = (; round_digits = 2, additional_metrics = (; height = r->SoleLogics.height(antecedent(r)))), 
    syntaxstring_kwargs = (; threshold_digits = 2), 
    variable_names_map=variable_names
);

┌ Info: Load dataset...
└ @ Main /home/paso/Documents/Aclai/audio-rules2024/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X43sZmlsZQ==.jl:53


▣ (mean(skwns) ≥ 17.49) ∧ (min(mel7=794Hz) ≥ -5.04) ∧ (min(mel16=1802Hz) ≥ -5.63) ∧ (3ac(mel10=1074Hz) ≥ 0.01) ∧ (med(flux) ≥ 1251.61) ∧ (min(mel9=975Hz) < -3.77) ∧ (std(decrs) < 6.8) ∧ (3ac(mel18=2106Hz) ≥ 0.02)  ↣  Healthy : (ninstances = 86, ncovered = 9, coverage = 0.1, confidence = 0.67, lift = 1.27, natoms = 8, height = 7)
▣ (mean(skwns) < 17.49) ∧ (min(mel9=975Hz) ≥ -6.03) ∧ (mean(mel23=3037Hz) < -6.36)  ↣  URTI : (ninstances = 86, ncovered = 16, coverage = 0.19, confidence = 0.94, lift = 1.97, natoms = 3, height = 2)


### URTI
scale: mel_htk with mfcc

In [17]:
# experiment = :Pneumonia
# experiment = :Bronchiectasis
# experiment = :COPD
experiment = :URTI
# experiment = :Bronchiolitis

# scale = :semitones
scale = :mel_htk

# featset = ()
featset = (:mfcc,)

audioparams = (
    sr = sr,
    nfft = 512,
    mel_scale = scale, # :mel_htk, :mel_slaney, :erb, :bark, :semitones, :tuned_semitones
    mel_nbands = scale == :semitones ? 14 : 26,
    mfcc_ncoeffs = scale == :semitones ? 7 : 13,
    mel_freqrange = (300, round(Int, sr / 2)),
    mel_dbscale = :mfcc in featset ? false : true,
    audio_norm = true,
)

@assert experiment in avail_exp "Unknown type of experiment: $experiment."
ds_path = "/datasets/respiratory_Healthy_" * String(experiment)
filename = "/datasets/itadata2024_" * String(experiment) * "_files"

destpath = "results/propositional/$scale"
:mfcc in featset ? destpath *= "_mfcc/" : destpath *= "/"
jld2file = destpath * "/itadata2024_" * String(experiment) * "_" * String(scale) * ".jld2"
dsfile = destpath * "/ds_test_" * String(experiment) * "_" * String(scale) * ".jld2"

d = jldopen(string((@__DIR__), ds_path, ".jld2"))
x, y = d["dataframe_validated"]
@assert x isa DataFrame
close(d)

freq = round.(Int, afe(x[1, :audio]; featset=(:get_only_freqs), audioparams...))

variable_names = vcat([
    vcat(
        ["\e[$(color_code[:yellow])m$j(mel$i=$(freq[i])Hz)\e[0m" for i in 1:audioparams.mel_nbands],
        :mfcc in featset ? ["\e[$(color_code[:red])m$j(mfcc$i)\e[0m" for i in 1:audioparams.mfcc_ncoeffs] : String[],
        :f0 in featset ? ["\e[$(color_code[:green])m$j(f0)\e[0m"] : String[],
        "\e[$(color_code[:cyan])m$j(cntrd)\e[0m", "\e[$(color_code[:cyan])m$j(crest)\e[0m",
        "\e[$(color_code[:cyan])m$j(entrp)\e[0m", "\e[$(color_code[:cyan])m$j(flatn)\e[0m", "\e[$(color_code[:cyan])m$j(flux)\e[0m",
        "\e[$(color_code[:cyan])m$j(kurts)\e[0m", "\e[$(color_code[:cyan])m$j(rllff)\e[0m", "\e[$(color_code[:cyan])m$j(skwns)\e[0m",
        "\e[$(color_code[:cyan])m$j(decrs)\e[0m", "\e[$(color_code[:cyan])m$j(slope)\e[0m", "\e[$(color_code[:cyan])m$j(sprd)\e[0m"
    )
    for j in catch9_f
]...)

@info("Load dataset...")
d = jldopen(dsfile)
X_test = d["X_test"]
y_test = d["y_test"]
close(d)
d = jldopen(jld2file)
sole_dt = d["sole_dt"]
close(d)

interesting_rules = listrules(sole_dt,
	min_lift = 1.0,
	# min_lift = 2.0,
	min_ninstances = 0,
	min_coverage = 0.10,
	normalize = true,
    variable_names_map=variable_names
);
map(r->(consequent(r), readmetrics(r)), interesting_rules)

readmetrics.(interesting_rules)

printmodel.(
    sort(interesting_rules, by = readmetrics); 
    show_metrics = (; round_digits = 2, additional_metrics = (; height = r->SoleLogics.height(antecedent(r)))), 
    syntaxstring_kwargs = (; threshold_digits = 2), 
    variable_names_map=variable_names
);

┌ Info: Load dataset...
└ @ Main /home/paso/Documents/Aclai/audio-rules2024/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X45sZmlsZQ==.jl:53


▣ (min(mfcc2) ≥ 1.78) ∧ (mean(mfcc8) < 0.62) ∧ (med(mfcc9) < 0.3) ∧ (med(mfcc7) ≥ -0.41) ∧ (qnt(mel10=1074Hz) ≥ 1.96) ∧ (mean(mfcc13) ≥ -0.13) ∧ (std(mfcc9) < 0.22) ∧ (med(mel13=1407Hz) ≥ 0.0)  ↣  URTI : (ninstances = 86, ncovered = 9, coverage = 0.1, confidence = 0.67, lift = 1.79, natoms = 8, height = 7)
▣ (min(mfcc2) ≥ 1.78) ∧ (mean(mfcc8) < 0.62) ∧ (med(mfcc9) ≥ 0.3) ∧ (med(mel4=558Hz) ≥ 0.0) ∧ (qnt(mel17=1950Hz) < 2.17)  ↣  URTI : (ninstances = 86, ncovered = 10, coverage = 0.12, confidence = 0.4, lift = 1.08, natoms = 5, height = 4)
▣ (min(mfcc2) < 1.78) ∧ (std(slope) ≥ 0.01) ∧ (std(mel20=2447Hz) < 0.0) ∧ (bsd(mfcc8) ≥ 4.5) ∧ (3ac(mfcc2) ≥ 0.01) ∧ (min(mfcc11) ≥ -1.07)  ↣  Healthy : (ninstances = 86, ncovered = 10, coverage = 0.12, confidence = 0.9, lift = 1.43, natoms = 6, height = 5)
▣ (min(mfcc2) ≥ 1.78) ∧ (mean(mfcc8) < 0.62) ∧ (med(mfcc9) < 0.3) ∧ (med(mfcc7) ≥ -0.41) ∧ (qnt(mel10=1074Hz) < 1.96) ∧ (med(mfcc13) < 0.2) ∧ (std(mel13=1407Hz) ≥ 0.0)  ↣  URTI : (ninstances = 86, 

### Bronchiolitis
scale: semitones without mfcc

In [18]:
# experiment = :Pneumonia
# experiment = :Bronchiectasis
# experiment = :COPD
# experiment = :URTI
experiment = :Bronchiolitis

scale = :semitones
# scale = :mel_htk

featset = ()
# featset = (:mfcc,)

audioparams = (
    sr = sr,
    nfft = 512,
    mel_scale = scale, # :mel_htk, :mel_slaney, :erb, :bark, :semitones, :tuned_semitones
    mel_nbands = scale == :semitones ? 14 : 26,
    mfcc_ncoeffs = scale == :semitones ? 7 : 13,
    mel_freqrange = (300, round(Int, sr / 2)),
    mel_dbscale = :mfcc in featset ? false : true,
    audio_norm = true,
)

@assert experiment in avail_exp "Unknown type of experiment: $experiment."
ds_path = "/datasets/respiratory_Healthy_" * String(experiment)
filename = "/datasets/itadata2024_" * String(experiment) * "_files"

destpath = "results/propositional/$scale"
:mfcc in featset ? destpath *= "_mfcc/" : destpath *= "/"
jld2file = destpath * "/itadata2024_" * String(experiment) * "_" * String(scale) * ".jld2"
dsfile = destpath * "/ds_test_" * String(experiment) * "_" * String(scale) * ".jld2"

d = jldopen(string((@__DIR__), ds_path, ".jld2"))
x, y = d["dataframe_validated"]
@assert x isa DataFrame
close(d)

freq = round.(Int, afe(x[1, :audio]; featset=(:get_only_freqs), audioparams...))

variable_names = vcat([
    vcat(
        ["\e[$(color_code[:yellow])m$j(mel$i=$(freq[i])Hz)\e[0m" for i in 1:audioparams.mel_nbands],
        :mfcc in featset ? ["\e[$(color_code[:red])m$j(mfcc$i)\e[0m" for i in 1:audioparams.mfcc_ncoeffs] : String[],
        :f0 in featset ? ["\e[$(color_code[:green])m$j(f0)\e[0m"] : String[],
        "\e[$(color_code[:cyan])m$j(cntrd)\e[0m", "\e[$(color_code[:cyan])m$j(crest)\e[0m",
        "\e[$(color_code[:cyan])m$j(entrp)\e[0m", "\e[$(color_code[:cyan])m$j(flatn)\e[0m", "\e[$(color_code[:cyan])m$j(flux)\e[0m",
        "\e[$(color_code[:cyan])m$j(kurts)\e[0m", "\e[$(color_code[:cyan])m$j(rllff)\e[0m", "\e[$(color_code[:cyan])m$j(skwns)\e[0m",
        "\e[$(color_code[:cyan])m$j(decrs)\e[0m", "\e[$(color_code[:cyan])m$j(slope)\e[0m", "\e[$(color_code[:cyan])m$j(sprd)\e[0m"
    )
    for j in catch9_f
]...)

@info("Load dataset...")
d = jldopen(dsfile)
X_test = d["X_test"]
y_test = d["y_test"]
close(d)
d = jldopen(jld2file)
sole_dt = d["sole_dt"]
close(d)

interesting_rules = listrules(sole_dt,
	min_lift = 1.0,
	# min_lift = 2.0,
	min_ninstances = 0,
	min_coverage = 0.10,
	normalize = true,
    variable_names_map=variable_names
);
map(r->(consequent(r), readmetrics(r)), interesting_rules)

readmetrics.(interesting_rules)

printmodel.(
    sort(interesting_rules, by = readmetrics); 
    show_metrics = (; round_digits = 2, additional_metrics = (; height = r->SoleLogics.height(antecedent(r)))), 
    syntaxstring_kwargs = (; threshold_digits = 2), 
    variable_names_map=variable_names
);

┌ Info: Load dataset...
└ @ Main /home/paso/Documents/Aclai/audio-rules2024/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X50sZmlsZQ==.jl:53


▣ (mean(kurts) < 5761.28) ∧ (med(mel6=845Hz) < -3.25) ∧ (mean(entrp) < 0.21) ∧ (max(mel1=357Hz) ≥ -2.47) ∧ (mean(skwns) < 15.58) ∧ (std(mel3=504Hz) ≥ 0.57) ∧ (qnt(kurts) < 2.18)  ↣  Bronchiolitis : (ninstances = 56, ncovered = 6, coverage = 0.11, confidence = 1.0, lift = 2.07, natoms = 7, height = 6)
▣ (mean(kurts) < 5761.28) ∧ (med(mel6=845Hz) < -3.25) ∧ (mean(entrp) ≥ 0.21) ∧ (min(skwns) < 3.35)  ↣  Bronchiolitis : (ninstances = 56, ncovered = 14, coverage = 0.25, confidence = 0.93, lift = 1.93, natoms = 4, height = 3)
▣ mean(kurts) ≥ 5761.28  ↣  Healthy : (ninstances = 56, ncovered = 15, coverage = 0.27, confidence = 0.87, lift = 1.67, natoms = 1, height = 0)


### Bronchiolitis
scale: semitones with mfcc

In [19]:
# experiment = :Pneumonia
# experiment = :Bronchiectasis
# experiment = :COPD
# experiment = :URTI
experiment = :Bronchiolitis

scale = :semitones
# scale = :mel_htk

# featset = ()
featset = (:mfcc,)

audioparams = (
    sr = sr,
    nfft = 512,
    mel_scale = scale, # :mel_htk, :mel_slaney, :erb, :bark, :semitones, :tuned_semitones
    mel_nbands = scale == :semitones ? 14 : 26,
    mfcc_ncoeffs = scale == :semitones ? 7 : 13,
    mel_freqrange = (300, round(Int, sr / 2)),
    mel_dbscale = :mfcc in featset ? false : true,
    audio_norm = true,
)

@assert experiment in avail_exp "Unknown type of experiment: $experiment."
ds_path = "/datasets/respiratory_Healthy_" * String(experiment)
filename = "/datasets/itadata2024_" * String(experiment) * "_files"

destpath = "results/propositional/$scale"
:mfcc in featset ? destpath *= "_mfcc/" : destpath *= "/"
jld2file = destpath * "/itadata2024_" * String(experiment) * "_" * String(scale) * ".jld2"
dsfile = destpath * "/ds_test_" * String(experiment) * "_" * String(scale) * ".jld2"

d = jldopen(string((@__DIR__), ds_path, ".jld2"))
x, y = d["dataframe_validated"]
@assert x isa DataFrame
close(d)

freq = round.(Int, afe(x[1, :audio]; featset=(:get_only_freqs), audioparams...))

variable_names = vcat([
    vcat(
        ["\e[$(color_code[:yellow])m$j(mel$i=$(freq[i])Hz)\e[0m" for i in 1:audioparams.mel_nbands],
        :mfcc in featset ? ["\e[$(color_code[:red])m$j(mfcc$i)\e[0m" for i in 1:audioparams.mfcc_ncoeffs] : String[],
        :f0 in featset ? ["\e[$(color_code[:green])m$j(f0)\e[0m"] : String[],
        "\e[$(color_code[:cyan])m$j(cntrd)\e[0m", "\e[$(color_code[:cyan])m$j(crest)\e[0m",
        "\e[$(color_code[:cyan])m$j(entrp)\e[0m", "\e[$(color_code[:cyan])m$j(flatn)\e[0m", "\e[$(color_code[:cyan])m$j(flux)\e[0m",
        "\e[$(color_code[:cyan])m$j(kurts)\e[0m", "\e[$(color_code[:cyan])m$j(rllff)\e[0m", "\e[$(color_code[:cyan])m$j(skwns)\e[0m",
        "\e[$(color_code[:cyan])m$j(decrs)\e[0m", "\e[$(color_code[:cyan])m$j(slope)\e[0m", "\e[$(color_code[:cyan])m$j(sprd)\e[0m"
    )
    for j in catch9_f
]...)

@info("Load dataset...")
d = jldopen(dsfile)
X_test = d["X_test"]
y_test = d["y_test"]
close(d)
d = jldopen(jld2file)
sole_dt = d["sole_dt"]
close(d)

interesting_rules = listrules(sole_dt,
	min_lift = 1.0,
	# min_lift = 2.0,
	min_ninstances = 0,
	min_coverage = 0.10,
	normalize = true,
    variable_names_map=variable_names
);
map(r->(consequent(r), readmetrics(r)), interesting_rules)

readmetrics.(interesting_rules)

printmodel.(
    sort(interesting_rules, by = readmetrics); 
    show_metrics = (; round_digits = 2, additional_metrics = (; height = r->SoleLogics.height(antecedent(r)))), 
    syntaxstring_kwargs = (; threshold_digits = 2), 
    variable_names_map=variable_names
);

┌ Info: Load dataset...
└ @ Main /home/paso/Documents/Aclai/audio-rules2024/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X52sZmlsZQ==.jl:53


▣ (max(mfcc2) < 6.36) ∧ (med(mfcc5) ≥ 0.11) ∧ (mean(crest) ≥ 134.78) ∧ (med(mfcc2) < 2.03)  ↣  Healthy : (ninstances = 56, ncovered = 7, coverage = 0.12, confidence = 1.0, lift = 1.81, natoms = 4, height = 3)
▣ (max(mfcc2) ≥ 6.36) ∧ (med(skwns) < 15.43)  ↣  Bronchiolitis : (ninstances = 56, ncovered = 9, coverage = 0.16, confidence = 0.78, lift = 1.74, natoms = 2, height = 1)
▣ (max(mfcc2) < 6.36) ∧ (med(mfcc5) < 0.11)  ↣  Healthy : (ninstances = 56, ncovered = 10, coverage = 0.18, confidence = 0.9, lift = 1.63, natoms = 2, height = 1)
▣ (max(mfcc2) < 6.36) ∧ (med(mfcc5) ≥ 0.11) ∧ (mean(crest) < 134.78) ∧ (qnt(mel6=845Hz) ≥ 1.85) ∧ (mean(mel4=599Hz) ≥ 0.0)  ↣  Bronchiolitis : (ninstances = 56, ncovered = 13, coverage = 0.23, confidence = 0.77, lift = 1.72, natoms = 5, height = 4)


### Bronchiolitis
scale: mel_htk without mfcc

In [20]:
# experiment = :Pneumonia
# experiment = :Bronchiectasis
# experiment = :COPD
# experiment = :URTI
experiment = :Bronchiolitis

# scale = :semitones
scale = :mel_htk

featset = ()
# featset = (:mfcc,)

audioparams = (
    sr = sr,
    nfft = 512,
    mel_scale = scale, # :mel_htk, :mel_slaney, :erb, :bark, :semitones, :tuned_semitones
    mel_nbands = scale == :semitones ? 14 : 26,
    mfcc_ncoeffs = scale == :semitones ? 7 : 13,
    mel_freqrange = (300, round(Int, sr / 2)),
    mel_dbscale = :mfcc in featset ? false : true,
    audio_norm = true,
)

@assert experiment in avail_exp "Unknown type of experiment: $experiment."
ds_path = "/datasets/respiratory_Healthy_" * String(experiment)
filename = "/datasets/itadata2024_" * String(experiment) * "_files"

destpath = "results/propositional/$scale"
:mfcc in featset ? destpath *= "_mfcc/" : destpath *= "/"
jld2file = destpath * "/itadata2024_" * String(experiment) * "_" * String(scale) * ".jld2"
dsfile = destpath * "/ds_test_" * String(experiment) * "_" * String(scale) * ".jld2"

d = jldopen(string((@__DIR__), ds_path, ".jld2"))
x, y = d["dataframe_validated"]
@assert x isa DataFrame
close(d)

freq = round.(Int, afe(x[1, :audio]; featset=(:get_only_freqs), audioparams...))

variable_names = vcat([
    vcat(
        ["\e[$(color_code[:yellow])m$j(mel$i=$(freq[i])Hz)\e[0m" for i in 1:audioparams.mel_nbands],
        :mfcc in featset ? ["\e[$(color_code[:red])m$j(mfcc$i)\e[0m" for i in 1:audioparams.mfcc_ncoeffs] : String[],
        :f0 in featset ? ["\e[$(color_code[:green])m$j(f0)\e[0m"] : String[],
        "\e[$(color_code[:cyan])m$j(cntrd)\e[0m", "\e[$(color_code[:cyan])m$j(crest)\e[0m",
        "\e[$(color_code[:cyan])m$j(entrp)\e[0m", "\e[$(color_code[:cyan])m$j(flatn)\e[0m", "\e[$(color_code[:cyan])m$j(flux)\e[0m",
        "\e[$(color_code[:cyan])m$j(kurts)\e[0m", "\e[$(color_code[:cyan])m$j(rllff)\e[0m", "\e[$(color_code[:cyan])m$j(skwns)\e[0m",
        "\e[$(color_code[:cyan])m$j(decrs)\e[0m", "\e[$(color_code[:cyan])m$j(slope)\e[0m", "\e[$(color_code[:cyan])m$j(sprd)\e[0m"
    )
    for j in catch9_f
]...)

@info("Load dataset...")
d = jldopen(dsfile)
X_test = d["X_test"]
y_test = d["y_test"]
close(d)
d = jldopen(jld2file)
sole_dt = d["sole_dt"]
close(d)

interesting_rules = listrules(sole_dt,
	min_lift = 1.0,
	# min_lift = 2.0,
	min_ninstances = 0,
	min_coverage = 0.10,
	normalize = true,
    variable_names_map=variable_names
);
map(r->(consequent(r), readmetrics(r)), interesting_rules)

readmetrics.(interesting_rules)

printmodel.(
    sort(interesting_rules, by = readmetrics); 
    show_metrics = (; round_digits = 2, additional_metrics = (; height = r->SoleLogics.height(antecedent(r)))), 
    syntaxstring_kwargs = (; threshold_digits = 2), 
    variable_names_map=variable_names
);

┌ Info: Load dataset...
└ @ Main /home/paso/Documents/Aclai/audio-rules2024/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X54sZmlsZQ==.jl:53


▣ (med(kurts) < 1994.17) ∧ (mean(mel6=710Hz) < -2.96) ∧ (mean(cntrd) ≥ 24.17)  ↣  Bronchiolitis : (ninstances = 56, ncovered = 6, coverage = 0.11, confidence = 1.0, lift = 2.24, natoms = 3, height = 2)
▣ (med(kurts) ≥ 1994.17) ∧ (max(mel9=975Hz) < -3.51)  ↣  Healthy : (ninstances = 56, ncovered = 7, coverage = 0.12, confidence = 0.86, lift = 1.55, natoms = 2, height = 1)
▣ (med(kurts) ≥ 1994.17) ∧ (max(mel9=975Hz) ≥ -3.51) ∧ (min(mel10=1074Hz) ≥ -4.72) ∧ (3ac(flux) ≥ 0.01)  ↣  Healthy : (ninstances = 56, ncovered = 8, coverage = 0.14, confidence = 0.62, lift = 1.13, natoms = 4, height = 3)
▣ (med(kurts) < 1994.17) ∧ (mean(mel6=710Hz) < -2.96) ∧ (mean(cntrd) < 24.17) ∧ (mean(mel13=1407Hz) ≥ -5.49) ∧ (mean(skwns) < 14.94) ∧ (bsm(mel22=2829Hz) < 11.0)  ↣  Bronchiolitis : (ninstances = 56, ncovered = 10, coverage = 0.18, confidence = 0.9, lift = 2.02, natoms = 6, height = 5)


### Bronchiolitis
scale: mel_htk with mfcc

In [21]:
# experiment = :Pneumonia
# experiment = :Bronchiectasis
# experiment = :COPD
# experiment = :URTI
experiment = :Bronchiolitis

# scale = :semitones
scale = :mel_htk

# featset = ()
featset = (:mfcc,)

audioparams = (
    sr = sr,
    nfft = 512,
    mel_scale = scale, # :mel_htk, :mel_slaney, :erb, :bark, :semitones, :tuned_semitones
    mel_nbands = scale == :semitones ? 14 : 26,
    mfcc_ncoeffs = scale == :semitones ? 7 : 13,
    mel_freqrange = (300, round(Int, sr / 2)),
    mel_dbscale = :mfcc in featset ? false : true,
    audio_norm = true,
)

@assert experiment in avail_exp "Unknown type of experiment: $experiment."
ds_path = "/datasets/respiratory_Healthy_" * String(experiment)
filename = "/datasets/itadata2024_" * String(experiment) * "_files"

destpath = "results/propositional/$scale"
:mfcc in featset ? destpath *= "_mfcc/" : destpath *= "/"
jld2file = destpath * "/itadata2024_" * String(experiment) * "_" * String(scale) * ".jld2"
dsfile = destpath * "/ds_test_" * String(experiment) * "_" * String(scale) * ".jld2"

d = jldopen(string((@__DIR__), ds_path, ".jld2"))
x, y = d["dataframe_validated"]
@assert x isa DataFrame
close(d)

freq = round.(Int, afe(x[1, :audio]; featset=(:get_only_freqs), audioparams...))

variable_names = vcat([
    vcat(
        ["\e[$(color_code[:yellow])m$j(mel$i=$(freq[i])Hz)\e[0m" for i in 1:audioparams.mel_nbands],
        :mfcc in featset ? ["\e[$(color_code[:red])m$j(mfcc$i)\e[0m" for i in 1:audioparams.mfcc_ncoeffs] : String[],
        :f0 in featset ? ["\e[$(color_code[:green])m$j(f0)\e[0m"] : String[],
        "\e[$(color_code[:cyan])m$j(cntrd)\e[0m", "\e[$(color_code[:cyan])m$j(crest)\e[0m",
        "\e[$(color_code[:cyan])m$j(entrp)\e[0m", "\e[$(color_code[:cyan])m$j(flatn)\e[0m", "\e[$(color_code[:cyan])m$j(flux)\e[0m",
        "\e[$(color_code[:cyan])m$j(kurts)\e[0m", "\e[$(color_code[:cyan])m$j(rllff)\e[0m", "\e[$(color_code[:cyan])m$j(skwns)\e[0m",
        "\e[$(color_code[:cyan])m$j(decrs)\e[0m", "\e[$(color_code[:cyan])m$j(slope)\e[0m", "\e[$(color_code[:cyan])m$j(sprd)\e[0m"
    )
    for j in catch9_f
]...)

@info("Load dataset...")
d = jldopen(dsfile)
X_test = d["X_test"]
y_test = d["y_test"]
close(d)
d = jldopen(jld2file)
sole_dt = d["sole_dt"]
close(d)

interesting_rules = listrules(sole_dt,
	min_lift = 1.0,
	# min_lift = 2.0,
	min_ninstances = 0,
	min_coverage = 0.10,
	normalize = true,
    variable_names_map=variable_names
);
map(r->(consequent(r), readmetrics(r)), interesting_rules)

readmetrics.(interesting_rules)

printmodel.(
    sort(interesting_rules, by = readmetrics); 
    show_metrics = (; round_digits = 2, additional_metrics = (; height = r->SoleLogics.height(antecedent(r)))), 
    syntaxstring_kwargs = (; threshold_digits = 2), 
    variable_names_map=variable_names
);

┌ Info: Load dataset...
└ @ Main /home/paso/Documents/Aclai/audio-rules2024/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X56sZmlsZQ==.jl:53


▣ (med(mfcc5) < 0.49) ∧ (max(flatn) ≥ 0.0) ∧ (med(decrs) ≥ -9.78)  ↣  Healthy : (ninstances = 56, ncovered = 10, coverage = 0.18, confidence = 0.8, lift = 1.72, natoms = 3, height = 2)
▣ (med(mfcc5) ≥ 0.49) ∧ (max(rllff) < 593.75) ∧ (mean(crest) < 133.96) ∧ (std(mel8=882Hz) < 0.0)  ↣  Bronchiolitis : (ninstances = 56, ncovered = 13, coverage = 0.23, confidence = 1.0, lift = 1.87, natoms = 4, height = 3)
